In [1]:
import pandas as pd
import numpy as np
import warnings
pd.pandas.set_option("display.max_columns",None)
warnings.filterwarnings('ignore')

In [2]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder, MinMaxScaler
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
house_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ETL_Housing_Data.csv")
house_df.head(20)

,Housing_Id,Address,sold_price,Summary,Housing_type,Year_built,heating_features,cooling_features,parking,Lot,bedrooms,bathrooms,full_bathrooms,total_interior_livable_area,Total spaces,Garage spaces,Region,Elementary School,elementary_school_score,elementary_school_distance,Middle School,middle_school_score,middle_school_distance,High School,high_school_score,high_school_disctance,Flooring,Appliances included,Laundry features,tax_assesses_value,annual_tax_amount,Listed_on,listed_price,Last Sold On,last_sold_price,City,Zip,H_State
0,1,1727 W 67th St,505000,"HURRY, HURRY.......Great house 3 bed and 2 bat...",SingleFamily,1926,Combination,"Wall/Window Unit(s), Evaporative Cooling, See ...","Detached Carport, Garage",4.047000e+03,3,2,2,872.000000,1,1,Los Angeles,Raymond Avenue Elementary School,3,0.800000,John Muir Middle School,2,1.100000,Augustus F. Hawkins High A Critical Design And...,2,1.300000,Missing,Missing,Inside,5.050000e+05,6253.000000,2019-10-16,525000,2019-08-30,3.280000e+05,Los Angeles,90047,CA
1,2,28093 Pine Ave,140000,'THE PERFECT CABIN TO FLIP! Strawberry deligh...,SingleFamily,1958,Forced air,Missing,0 spaces,9.147000e+03,2,3,1,1152.000000,0,0,Strawberry,Missing,5,1.152411,Missing,5,1.691593,Long Barn High School,6,10.100000,Missing,Missing,Missing,4.962700e+04,468.000000,2019-08-25,180000,2019-08-30,8.078537e+05,Strawberry,95375,CA
2,3,10750 Braddock Dr,1775000,Rare 2-story Gated 5 bedroom Modern Mediterran...,SingleFamily,1947,Central,Central Air,"Detached Carport, Driveway, Garage - Two Door",2.353383e+05,5,3,3,2612.000000,0,0,Culver City,Farragut Elementary School,9,0.200000,Culver City Middle School,7,0.200000,Culver City High School,8,0.200000,"Wood, Tile",Dishwasher,"Stackable, Upper Level",1.775000e+06,20787.000000,2019-10-24,1895000,2016-08-30,1.500000e+06,Culver City,90230,CA
3,5,922 E Channel St,221000,"922 E Channel St, Stockton, CA 95202 is a sing...",SingleFamily,1905,Wall Furnace,Window Unit(s),Carport,3.576000e+03,3,2,2,1311.000000,0,0,Stockton,Missing,5,1.152411,Missing,5,1.691593,Stagg Senior High School,2,3.300000,"Wood, Tile",Range / Oven,Inside Area,2.080800e+05,2531.000000,2019-10-30,224900,2017-12-28,2.000000e+05,Stockton,95202,CA
4,7,9013 N Valley Rd,480000,"9013 N Valley Rd, Taylorsville, CA 95983 is a ...",SingleFamily,2005,"Forced air, Propane / Butane, Solar",Other,"Carport, Garage - Attached, Covered",1.771150e+06,3,2,2,2519.000000,4,4,Taylorsville,Indian Valley Elementary School,4,6.300000,Missing,5,1.691593,Greenville Junior/Senior High School,2,6.300000,"Carpet, Hardwood, Linoleum / Vinyl","Dishwasher, Dryer, Garbage disposal, Microwave...",Missing,7.863118e+05,9956.843817,2019-10-18,499000,2019-08-30,8.078537e+05,Taylorsville,95983,CA
5,8,225 26th Ave #2,1590000,New Refinished Bathrooms! Come take a NEW look...,Condo,2001,Missing,Missing,"Attached, Enclosed, Garage Door Opener, Interi...",2.353383e+05,4,3,3,1601.000000,1,1,San Francisco,Alamo Elementary School,8,0.300000,Presidio Middle School,8,0.300000,Washington (George) High School,7,0.500000,"Carpet, Wood","Dishwasher, Disposal, Range Hood, Ice Maker",Laundry Closet,1.087088e+06,13793.000000,2019-07-18,1650000,2019-08-30,8.078537e+05,San Francisco,94121,CA
6,9,612 Valencia Rd,1275000,Sprawling one level home on 1.5 acres with lev...,SingleFamily,1973,"Gas, Heating - 2+ Zones, Wall Furnace, Central...",None,"Garage, Garage - Attached",6.621120e+04,0,2,2,2123.000000,0,0,Aptos,Valencia Elementary School,6,0.600000,Missing,5,1.691593,Aptos High School,7,1.500000,"Tile, Carpet","Dryer, Range / Oven, Washer","Washer / Dryer, Inside, In Utility Room",1.677670e+05,1909.000000,2019-10-28,1050000,2019-08-30,8.078537e+05,Aptos,95003,CA
7,10,391 Kohner Ct,1310000,Striking home in one of the best neighborhoods...,SingleFamily,1953,Forced Air,Central AC,"Garage, Garage - Attached, Covered",7.570000e+03,4,2,2,1579.000000,2,2,Santa Clara,Westwood Elementary School,7,0.400000,Buchser Middle School,5,0.900000,Santa Clara High School,6,1.900000,

In [5]:
# Drop the non-beneficial columns
house_df = house_df.drop(["Housing_Id","Address", "Last Sold On", "H_State", 'Summary', 'Elementary School', 'Middle School', 'High School', 'parking', 'Listed_on', 'Year_built','Lot'],1)

In [6]:
# Generate our categorical variable lists
house_categorical = house_df.dtypes[house_df.dtypes == "object"].index.tolist()
house_categorical

['Housing_type',
 'heating_features',
 'cooling_features',
 'Region',
 'Flooring',
 'Appliances included',
 'Laundry features',
 'City']

In [7]:
# Determine the number of unique values in each column.
house_df[house_categorical].nunique()

Housing_type            140
heating_features       1572
cooling_features        528
Region                  931
Flooring               1156
Appliances included    4458
Laundry features       1637
City                    910
dtype: int64

In [8]:
# Look at Housing_type value counts for binning
housetype_counts = house_df.Housing_type.value_counts()
housetype_counts 

SingleFamily                                                                                           25814
Condo                                                                                                   7178
Townhouse                                                                                               1805
MobileManufactured                                                                                       809
MultiFamily                                                                                              756
                                                                                                       ...  
Mid-Rise (4-8), End Unit                                                                                   1
Detached, 4+ Story                                                                                         1
Unit Below, Flat, Unit Above                                                                               1
Luxury, Mid-Rise (4

In [9]:
# Determine which values to replace
replace_housetype = list(housetype_counts[housetype_counts < 750].index)

# Replace in dataframe
for app in replace_housetype:
    house_df.Housing_type = house_df.Housing_type.replace(app,"Other")
    
# Check to make sure binning was successful
house_df.Housing_type.value_counts()

SingleFamily          25814
Condo                  7178
Townhouse              1805
Other                   882
MobileManufactured      809
MultiFamily             756
Name: Housing_type, dtype: int64

In [10]:
# Look at heating_features value counts for binning
heatingfeatures_counts = house_df.heating_features.value_counts()
heatingfeatures_counts

Central                                                                9325
Missing                                                                4898
Central Forced Air                                                     3066
Forced Air                                                             1944
Central Forced Air - Gas                                               1762
                                                                       ... 
Propane, Solar, Fireplace(s), Forced Air                                  1
Central, Natural Gas, ENERGY STAR Qualified Equipment, Fireplace(s)       1
Other, Electric, Other, Wood / Pellet                                     1
Mini Split                                                                1
Forced air, Stove, Electric, Solar, Wood / Pellet                         1
Name: heating_features, Length: 1572, dtype: int64

In [11]:
# Determine which values to replace
replace_heatingfeature = list(heatingfeatures_counts[heatingfeatures_counts < 1760].index)

# Replace in dataframe
for app in replace_heatingfeature:
    house_df.heating_features = house_df.heating_features.replace(app,"Other")
    
# Check to make sure binning was successful
house_df.heating_features.value_counts()

Other                       16249
Central                      9325
Missing                      4898
Central Forced Air           3066
Forced Air                   1944
Central Forced Air - Gas     1762
Name: heating_features, dtype: int64

In [12]:
# Look at cooling_features value counts for binning
coolingfeatures_counts = house_df.cooling_features.value_counts()
coolingfeatures_counts


Missing                                                  8843
Central Air                                              8645
None                                                     7427
Central AC                                               3143
Central                                                  2663
                                                         ... 
None, Attic Fan                                             1
AC Central, Ceiling Fans, Heat Pump, Evap Guest House       1
Heat Pump, Electric                                         1
AC Central, AC Wall, Ceiling Fans                           1
High Efficiency, SEER Rated 16+                             1
Name: cooling_features, Length: 528, dtype: int64

In [13]:
# Determine which values to replace
replace_coolingfeature = list(coolingfeatures_counts[coolingfeatures_counts < 2660].index)

# Replace in dataframe
for app in replace_coolingfeature:
    house_df.cooling_features = house_df.cooling_features.replace(app,"Other")
    
# Check to make sure binning was successful
house_df.cooling_features.value_counts()

Missing        8843
Central Air    8645
None           7427
Other          6523
Central AC     3143
Central        2663
Name: cooling_features, dtype: int64

In [14]:
# Look at Region value counts for binning
region_counts = house_df.Region.value_counts()
region_counts

Los Angeles      8721
San Francisco    4888
San Jose         4346
San Mateo         514
Sunnyvale         510
                 ... 
Belden              1
Fort Bidwell        1
Redding             1
Lost Hills          1
Elk Grove           1
Name: Region, Length: 931, dtype: int64

In [15]:
# Determine which values to replace
region_feature = list(region_counts[region_counts < 500].index)

# Replace in dataframe
for app in region_feature:
    house_df.Region = house_df.Region.replace(app,"Other")
    
# Check to make sure binning was successful
house_df.Region.value_counts()

Other            18265
Los Angeles       8721
San Francisco     4888
San Jose          4346
San Mateo          514
Sunnyvale          510
Name: Region, dtype: int64

In [16]:
# Look at Flooring value counts for binning
flooring_counts = house_df.Flooring.value_counts()
flooring_counts

Missing                         9106
Wood                            3809
Hardwood                        1920
Tile, Wood                      1518
Laminate                        1475
                                ... 
Vinyl, Softwood, Wood              1
Carpet, Mixed, Marble, Stone       1
Hardwood, Granite, Carpet          1
Ceramic Tile, Concrete             1
Stone, Linoleum/Vinyl              1
Name: Flooring, Length: 1156, dtype: int64

In [17]:
# Determine which values to replace
flooring_feature = list(flooring_counts[flooring_counts < 1470].index)

# Replace in dataframe
for app in flooring_feature:
    house_df.Flooring = house_df.Flooring.replace(app,"Other")
    
# Check to make sure binning was successful
house_df.Flooring.value_counts()

Other         19416
Missing        9106
Wood           3809
Hardwood       1920
Tile, Wood     1518
Laminate       1475
Name: Flooring, dtype: int64

In [18]:
# Look at Appliances included value counts for binning
appliance_counts = house_df['Appliances included'].value_counts()
appliance_counts

Missing                                                                                                                                      10964
Dryer, Washer                                                                                                                                 1144
Dishwasher, Dryer, Garbage disposal, Microwave, Range / Oven, Refrigerator, Washer                                                            1077
Dishwasher, Dryer, Freezer, Garbage disposal, Microwave, Range / Oven, Refrigerator, Washer                                                    859
Dishwasher                                                                                                                                     793
                                                                                                                                             ...  
Dishwasher, Disposal, Refrigerator, Gas Oven, Gas Cooktop, Range Hood                                                 

In [19]:
# Determine which values to replace
appliance_feature = list(appliance_counts[appliance_counts < 790].index)

# Replace in dataframe
for app in appliance_feature:
    house_df['Appliances included'] = house_df['Appliances included'].replace(app,"Other")
    
# Check to make sure binning was successful
house_df['Appliances included'].value_counts()

Other                                                                                          22407
Missing                                                                                        10964
Dryer, Washer                                                                                   1144
Dishwasher, Dryer, Garbage disposal, Microwave, Range / Oven, Refrigerator, Washer              1077
Dishwasher, Dryer, Freezer, Garbage disposal, Microwave, Range / Oven, Refrigerator, Washer      859
Dishwasher                                                                                       793
Name: Appliances included, dtype: int64

In [20]:
# Look at Laundry features value counts for binning
laundry_counts = house_df['Laundry features'].value_counts()
laundry_counts

Missing                                                                                        12070
Inside                                                                                          3122
In Garage                                                                                       2729
Laundry Closet                                                                                  1656
Laundry Room                                                                                    1399
                                                                                               ...  
Gas & Electric Dryer Hookup, Laundry Closet, Laundry Room, Inside, Stackable, Washer Hookup        1
Room, Basement                                                                                     1
Electric Dryer Hookup, Propane Dryer Hookup, In Garage                                             1
Area, Gas & Electric Dryer Hookup                                                          

In [21]:
# Determine which values to replace
laundry_feature = list(laundry_counts[laundry_counts < 1390].index)

# Replace in dataframe
for app in laundry_feature:
    house_df['Laundry features'] = house_df['Laundry features'].replace(app,"Other")
    
# Check to make sure binning was successful
house_df['Laundry features'].value_counts()

Other             16268
Missing           12070
Inside             3122
In Garage          2729
Laundry Closet     1656
Laundry Room       1399
Name: Laundry features, dtype: int64

In [22]:
# Look at City value counts for binning
city_counts = house_df['City'].value_counts()
city_counts

Los Angeles      8642
San Francisco    4889
San Jose         4196
San Mateo         649
Santa Clara       634
                 ... 
Macdoel             1
Phillipsville       1
Bard                1
Perris              1
Elk Grove           1
Name: City, Length: 910, dtype: int64

In [23]:
# Determine which values to replace
city_feature = list(city_counts[city_counts < 630].index)

# Replace in dataframe
for app in city_feature:
    house_df['City'] = house_df['City'].replace(app,"Other")
    
# Check to make sure binning was successful
house_df['City'].value_counts()

Other            18234
Los Angeles       8642
San Francisco     4889
San Jose          4196
San Mateo          649
Santa Clara        634
Name: City, dtype: int64

In [24]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(house_df[house_categorical]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(house_categorical)
encode_df.head()

,Housing_type_Condo,Housing_type_MobileManufactured,Housing_type_MultiFamily,Housing_type_Other,Housing_type_SingleFamily,Housing_type_Townhouse,heating_features_Central,heating_features_Central Forced Air,heating_features_Central Forced Air - Gas,heating_features_Forced Air,heating_features_Missing,heating_features_Other,cooling_features_Central,cooling_features_Central AC,cooling_features_Central Air,cooling_features_Missing,cooling_features_None,cooling_features_Other,Region_Los Angeles,Region_Other,Region_San Francisco,Region_San Jose,Region_San Mateo,Region_Sunnyvale,Flooring_Hardwood,Flooring_Laminate,Flooring_Missing,Flooring_Other,"Flooring_Tile, Wood",Flooring_Wood,Appliances included_Dishwasher,"Appliances included_Dishwasher, Dryer, Freezer, Garbage disposal, Microwave, Range / Oven, Refrigerator, Washer","Appliances included_Dishwasher, Dryer, Garbage disposal, Microwave, Range / Oven, Refrigerator, Washer","Appliances included_Dryer, Washer",Appliances included_Missing,Appliances included_Other,Laundry features_In Garage,Laundry features_Inside,Laundry features_Laundry Closet,Laundry features_Laundry Room,Laundry features_Missing,Laundry features_Other,City_Los Angeles,City_Other,City_San Francisco,City_San Jose,City_San Mateo,City_Santa Clara
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [25]:
# Merge one-hot encoded features and drop the originals
house_df = house_df.merge(encode_df,left_index=True, right_index=True)
house_df = house_df.drop(house_categorical,1)
house_df.head()

,sold_price,bedrooms,bathrooms,full_bathrooms,total_interior_livable_area,Total spaces,Garage spaces,elementary_school_score,elementary_school_distance,middle_school_score,middle_school_distance,high_school_score,high_school_disctance,tax_assesses_value,annual_tax_amount,listed_price,last_sold_price,Zip,Housing_type_Condo,Housing_type_MobileManufactured,Housing_type_MultiFamily,Housing_type_Other,Housing_type_SingleFamily,Housing_type_Townhouse,heating_features_Central,heating_features_Central Forced Air,heating_features_Central Forced Air - Gas,heating_features_Forced Air,heating_features_Missing,heating_features_Other,cooling_features_Central,cooling_features_Central AC,cooling_features_Central Air,cooling_features_Missing,cooling_features_None,cooling_features_Other,Region_Los Angeles,Region_Other,Region_San Francisco,Region_San Jose,Region_San Mateo,Region_Sunnyvale,Flooring_Hardwood,Flooring_Laminate,Flooring_Missing,Flooring_Other,"Flooring_Tile, Wood",Flooring_Wood,Appliances included_Dishwasher,"Appliances included_Dishwasher, Dryer, Freezer, Garbage disposal, Microwave, Range / Oven, Refrigerator, Washer","Appliances included_Dishwasher, Dryer, Garbage disposal, Microwave, Range / Oven, Refrigerator, Washer","Appliances included_Dryer, Washer",Appliances included_Missing,Appliances included_Other,Laundry features_In Garage,Laundry features_Inside,Laundry features_Laundry Closet,Laundry features_Laundry Room,Laundry features_Missing,Laundry features_Other,City_Los Angeles,City_Other,City_San Francisco,City_San Jose,City_San Mateo,City_Santa Clara
0,505000,3,2,2,872.0,1,1,3,0.800000,2,1.100000,2,1.3,5.050000e+05,6253.000000,525000,3.280000e+05,90047,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,140000,2,3,1,1152.0,0,0,5,1.152411,5,1.691593,6,10.1,4.962700e+04,468.000000,180000,8.078537e+05,95375,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1775000,5,3,3,2612.0,0,0,9,0.200000,7,0.200000,8,0.2,1.775000e+06,20787.000000,1895000,1.500000e+06,90230,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,221000,3,2,2,1311.0,0,0,5,1.152411,5,1.691593,2,3.3,2.080800e+05,2531.000000,224900,2.000000e+05,95202,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,480000,3,2,2,2519.0,4,4,4,6.300000,5,1.691593,2,6.3,7.863118e+05,9956.843817,499000,8.078537e+05,95983,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [26]:
# Split our preprocessed data into our features and target arrays
y = house_df["sold_price"]
X = house_df.drop(columns=["sold_price"], axis = 1)

In [27]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state =42)

In [28]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [29]:
model = LinearRegression()
model.fit(X_train_scaled,y_train)

LinearRegression()

In [30]:
y_pred = model.predict(X_test_scaled)

In [31]:
model.score(X_train_scaled,y_train)

0.8218749747787905

In [32]:
model.score(X_test_scaled,y_test)

0.46105604120201615

In [33]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost.sklearn import XGBRegressor

In [41]:
random_model = RandomForestRegressor(n_estimators=100, random_state=1)
random_model.fit(X_train_scaled,y_train )
y_pred = random_model.predict(X_test_scaled)

In [42]:
random_model.score(X_train_scaled,y_train)

0.9802301337477444

In [43]:
random_model.score(X_test_scaled,y_test)

0.6242720255803786

In [51]:
gradient_model = GradientBoostingRegressor(n_estimators=100, random_state=1)
gradient_model.fit(X_train_scaled,y_train )
y_pred = gradient_model.predict(X_test_scaled)

In [52]:
gradient_model.score(X_train_scaled,y_train)

0.8982506340121561

In [53]:
gradient_model.score(X_test_scaled,y_test)

0.6272120588886083

In [44]:
xgboost_model = XGBRegressor(max_depth=3, n_estimators=100, random_state=1)
xgboost_model.fit(X_train_scaled,y_train )
y_pred = xgboost_model.predict(X_test_scaled)

[10:24:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [46]:
xgboost_model.score(X_train_scaled,y_train)

0.892624092852123

In [47]:
xgboost_model.score(X_test_scaled,y_test)

0.6226810028804096